# 01 Mar ChFinAnn分句
之前发现有长句子>128的被截断了
将句子重新拆分，<128的合并

In [1]:
import json
def read_file(path):
    res = []
    with open(path,'r',encoding='utf-8') as f:
        return json.load(f)

# load dataset & data exploration

In [45]:
#train = read_file("./typed_train.json")
#dev = read_file("./typed_dev.json")
test = read_file("./typed_test.json")

In [36]:
num_sent = []
for i in range(len(test)):
    #for sent in train[i][1]['sentences']:
    num_sent.append(len(test[i][1]['sentences']))
    

In [37]:
import numpy as bp
bp.mean(num_sent)

21.015605493133585

In [82]:
max(num_sent)

832

In [7]:
def sent_seg(
    text,
    special_seg_indicators=None,
    lang="zh",
    punctuations=None,
    quotation_seg_mode=True,
) -> list:
    """
    cut texts into sentences (in chinese language).
    Args:
        text <str>: texts ready to be cut
        special_seg_indicators <list>: some special segment indicators and
            their replacement ( [indicator, replacement] ), in baike data,
            this argument could be `[('###', '\n'), ('%%%', ' '), ('%%', ' ')]`
        lang <str>: languages that your corpus is, support `zh` for Chinese
            and `en` for English now.
        punctuations <set>: you can split the texts by specified punctuations.
            texts will not be splited by `;`, so you can specify them by your own.
        quotation_seg_mode <bool>: if True, the quotations will be regarded as a
            part of the former sentence.
            e.g. `我说：“翠花，上酸菜。”，她说：“欸，好嘞。”`
            the text will be splited into
            ['我说：“翠花，上酸菜。”，', '她说：“欸，好嘞。”'], other than
            ['我说：“翠花，上酸菜。', '”，她说：“欸，好嘞。”']
    Rrturns:
        <list>: a list of strings, which are splited sentences.
    """
    # if texts are not in string format, raise an error
    if not isinstance(text, str):
        raise ValueError

    # if the text is empty, return a list with an empty string
    if len(text) == 0:
        return []

    text_return = text

    # segment on specified indicators
    # special indicators standard, like [('###', '\n'), ('%%%', '\t'), ('\s', '')]
    if special_seg_indicators:
        for indicator in special_seg_indicators:
            text_return = re.sub(indicator[0], indicator[1], text_return)

    if lang == "zh":
        punkt = {"。", "？", "！", "…"}
    elif lang == "en":
        punkt = {".", "?", "!"}
    if punctuations:
        punkt = punkt | punctuations

    if quotation_seg_mode:
        text_return = re.sub(
            "([%s]+[’”`'\"]*)" % ("".join(punkt)), "\\1\n", text_return
        )
    else:
        text_return = re.sub("([{}])".format("".join(punkt)), "\\1\n", text_return)

    # drop sentences with no length
    return [
        s.strip()
        for s in filter(
            lambda x: len(x.strip()) == 1
            and x.strip() not in punkt
            or len(x.strip()) > 0,
            text_return.split("\n"),
        )
    ]


def stat_sent_len(filepath):
    num_sents = []
    sent_len = []
    for d in load_line_json_iterator(filepath):
        sents = sent_seg(d["text"])
        num_sents.append(len(sents))
        lens = [len(sent) for sent in sents]
        sent_len.extend(lens)
        # if min(lens) < 5:
        #     print("================= raw text =================")
        #     print(d["text"])
        #     print("================= processed text =================")
        #     print("\n".join(filter(lambda x: len(x) < 5, sents)))
        #     breakpoint()
    sent_len_counter = Counter(sent_len)
    print(
        (
            f"num_sents: min: {min(num_sents)}, median: {median(num_sents)}, max: {max(num_sents)}\n"
            f"sent_len: min: {min(sent_len)}, median: {median(sent_len)}, max: {max(sent_len)}"
            f"{sent_len_counter.most_common()}"
        )
    )

# qy:将短句子合并为每句总长不超过128
def reorganise_sents(sents, max_seq_len, concat=False, final_cut=False, concat_str=" "):
    # qy:concat 是否合并句子
    new_sents = []
    group = ""
    for sent in sents:
        if len(sent) + len(group) < max_seq_len:
            if concat:
                if len(group) > 1 and "\u4e00" <= group[-1] <= "\u9fa5":
                    group += concat_str + sent
                else:
                    group += sent
            else:
                new_sents.append(sent)
        else:
            if len(group) > 0:
                new_sents.append(group)
                group = ""
            if len(sent) > max_seq_len:
                if final_cut:
                    group = sent[:max_seq_len]
                else:
                    sent_splits = sent_seg(sent, punctuations={"，", "、"})
                    reorg_sent_splits = reorganise_sents(
                        sent_splits, max_seq_len, concat=True, final_cut=True
                    )
                    new_sents.extend(reorg_sent_splits)
            else:
                group = sent
    if len(group) > 0:
        new_sents.append(group)
    return [s.strip() for s in filter(lambda x: len(x) > 0, new_sents)]


In [37]:
train[2122][1]['sentences']

['证券简称：大连热电证券代码：600719编号：临2012-022',
 '大连热电股份有限公司股份冻结提示性公告',
 '本公司及董事会全体成员保证公告内容的真实、准确和完整，对公告的虚假记载、误导性陈述或者重大遗漏负连带责任。',
 '公司曾在2012年半年度报告中披露控股股东大连市热电集团有限公司（持有本公司66566900股股份,占总股本的32.91%，以下简称“热电集团”）持有本公司的8000000股无限售流通股股份（占公司总股本的3.95%）被司法冻结。',
 '经向中国证券登记结算有限责任公司上海分公司查询，并经热电集团进一步查证获悉，系热电集团与大连振发房地产开发有限公司因热电集团组建前原春海热电有限公司动迁事宜发生动迁承包合同纠纷被辽宁省大连市中级人民法院司法冻结。',
 '冻结期限为2012年3月30日至2014年3月29日。',
 '特此公告。',
 '大连热电股份有限公司董事会',
 '二○一二年十二月六日']

In [39]:
text = train[1143][1]['sentences']
text = '。'.join(text[:3])+''.join(text[3:])
text

'股票代码：601002股票简称：晋亿实业公告编号：临2011-020号。晋亿实业股份有限公司关于股东减持股份的公告。本公司董事会及全体董事保证本公告内容不存在任何虚假记载、误导性陈述或者重大遗漏，并对其内容的真实性、准确性和完整性承担个别及连带责任。2011年5月11日，本公司接到第二大股东丁建中函告，2011年5月10日丁建中通过上海证劵交易所大宗交易方式减持晋亿实业股份5500000股，占公司股份总数的0.74%。本次减持后，丁建中尚持有晋亿实业股份36900100股，均为无限售条件流通股份，占公司股份总数的4.997%。按照有关规定,丁建中委托本公司代为履行公告义务。特此公告。晋亿实业股份有限公司董事会2011年5月12日'

In [50]:
import re
import sys
import json
from collections import Counter, defaultdict
from statistics import median
def convert_sents(text):
    #text = '。'.join(text1[:3])+''.join(text1[3:])
    sents = sent_seg(text, punctuations={"；"}) # qy:sentence segmentation
    #sents = reorganise_sents(sents, max_seq_len = 128, concat=True) # qy:合并短句
            # sents = d['map_sentences']
            # sentence length filtering
    sents = list(filter(lambda x: len(x) >= 5, sents)) # qy:去除<5个字的句子
    return sents

In [42]:
def convert_data(dataset):
    for data in dataset:
        data[1]['sentences'] = convert_sents(data[1]['sentences'])

In [30]:
convert_data(train)


In [34]:
train[113]

['SH600477_2009-04-15_51238506',
 {'sentences': ['证券代码：600477证券简称：杭萧钢构编号：临2009-009。',
   '浙江杭萧钢构股份有限公司关于股东减持公司公司股份的公告。',
   '本公司董事会及全体董事保证本公告内容不存在任何虚假记载、误导性陈述或者重大遗漏，并对其内容的真实性、准确性和完整性承担个别及连带责任。',
   '2009年4月13日，本公司接股东潘金水通知，截止2009年4月13日，潘金水先生通过上海证券交易所交易系统累计出售其所持本公司无限售条件流通股3216275股，占本公司总股本的1.30%。',
   '本次权益变动后，潘金水先生尚持有本公司股份20252000股，占本公司总股本的8.18%，其中有限售条件流通股11089584股，无限售条件流通股9162416股。',
   '公司董事会郑重提醒广大投资者注意投资风险，理性投资。',
   '《上海证券报》为公司指定的信息披露报纸，公司将严格按照有关法律法规的规定和要求，真实、准确、完整、及时做好信息披露工作。',
   '特此公告。',
   '浙江杭萧钢构股份有限公司董事会2009年4月14日'],
  'ann_valid_mspans': ['600477',
   '杭萧钢构',
   '杭萧钢构股份有限公司',
   '2009年4月13日',
   '潘金水',
   '3216275股',
   '9162416股'],
  'ann_valid_dranges': [[0, 5, 11],
   [0, 16, 20],
   [1, 2, 12],
   [3, 0, 10],
   [3, 17, 20],
   [3, 25, 35],
   [3, 36, 39],
   [3, 72, 80],
   [4, 8, 11],
   [4, 73, 81],
   [8, 2, 12]],
  'ann_mspan2dranges': {'600477': [[0, 5, 11]],
   '杭萧钢构': [[0, 16, 20]],
   '杭萧钢构股份有限公司': [[1, 2, 12], [8, 2, 12]],
   '2009年4月13日': [[3, 0, 10], [3, 25, 35

In [32]:
num_sent = []
for i in range(len(train)):
    #for sent in train[i][1]['sentences']:
    num_sent.append(len(train[i][1]['sentences']))
bp.mean(num_sent)    

13.804775280898877

In [53]:
convert_data(test)
convert_data(dev)

In [54]:
test[2]

['SZ002239_2018-09-08_1205425879',
 {'sentences': ['股票代码：002239证券简称：奥特佳公告编号：2018-090。奥特佳新能源科技股份有限公司关于公司股东股份质押的公告。本公司及董事会全体成员保证本次信息披露的内容真实、准确、完整，没有虚假记载、误导性陈述或重大遗漏。',
   '2018年9月7日，本公司接到公司股东北京天佑投资有限公司（以下简称“北京天佑”）通知，当日该公司将其所持有的本公司部分股份办理了股票质押手续，具体情况如下：1、股份质押的基本情况*北京天佑受本公司实际控制人张永明先生控制，',
   '与同受张永明先生控制的本公司股东江苏天佑金淦投资有限公司、西藏天佑投资有限公司构成一致行动关系。',
   '2、北京天佑股份被累计质押的情况截至本公告披露日，北京天佑共持有本公司股份325438596股，占本公司总股本的10.39%；此次质押的10000000股占其持有的本公司股份的3.07%，占公司总股本的0.32%；',
   '北京天佑所持有的本公司股份已累计被质押225000000股，占其所持有本公司股份的69.14%，占公司总股本的7.19%。',
   '截至本公告披露日，本公司实际控制人及其包括北京天佑在内的一致行动人共质押本公司股份287018581股，占其合计持有本公司741297531股股份的38.72%，占公司总股本的9.166%。3、备查文件申万宏源西部证券有限公司出具的《合并质押交割单》。',
   '特此公告。奥特佳新能源科技股份有限公司董事会二〇一八年九月八日'],
  'ann_valid_mspans': ['002239',
   '奥特佳',
   '奥特佳新能源科技股份有限公司',
   '2018年9月7日',
   '北京天佑投资有限公司',
   '江苏天佑金淦投资有限公司',
   '325438596股',
   '10.39%',
   '10000000股',
   '0.32%',
   '225000000股',
   '申万宏源西部证券有限公司'],
  'ann_valid_dranges': [[0, 5, 11],
   [0, 16, 19],
   [1, 0, 14],
   [4, 0, 9],
   [4, 19, 29],

In [55]:
def write_to_file(path,dataset):
    with open(path,"w",encoding="utf-8") as fout:
        for data in dataset:
            fout.write(json.dumps(data,ensure_ascii = False)+"\n")

write_to_file("typed_train_short.json",train)
write_to_file("typed_dev_short.json",dev)
write_to_file("typed_test_short.json",test)

### 查看分完之后的句子平均数量

In [62]:
num_sent = []
for i in range(len(train)):
    
    num_sent.append(len(train[i][1]['sentences']))
    if len(train[i][1]['sentences'])>64:
       print (i)

10470
20037


In [72]:
import numpy as np
np.mean(num_sent)

9.980220037453183

In [83]:
train[20037]

['SZ300298_2017-06-15_1203615530',
 {'sentences': ['证券代码：300298证券简称：三诺生物公告编号：2017-055',
   '三诺生物传感股份有限公司关于公司股权激励计划全部剩余股票期权与限制性股票回购注销完成的公告',
   '公司及董事会全体成员保证公告内容的真实、准确和完整，对公告的虚假记载、误导性陈述或者重大遗漏负连带责任。',
   '特别提示：',
   '1、本次回购注销的股票数量为989679股，占回购前公司总股本406026518股的0.24%，累计支付回购金额人民币8510173.49元。',
   '2、本次注销的股票期权数量为1646269份，占注销前公司总股本的0.49%。',
   '3、公司已于2017年6月14日在中国证券登记结算公司深圳分公司办理完毕前述限制性股票回购注销和股票期权注销业务。',
   '本次回购注销完成后，公司股份总数由406026518股变更为405036839股',
   '三诺生物传感股份有限公司（以下称“公司”或“三诺生物”）于2017年4月10日召开第三届董事会第五次会议，审议通过了《关于拟回购注销公司股权激励计划全部剩余股票期权与限制性股票的议案》，决定注销全部不符合行权条件的股票期权1646269份，回购注销全部不符合解锁条件的限制性股票824733股（含预留部分限制性股票），鉴于公司在本次限制性股票回购注销前已实施了2016年度权益分派方案（每10股派3.6元转增2股），本次实际应回购注销的限制性股份总额调整为989679股。',
   '公司已于2017年6月14日完成了上述限制性股票回购注销和股票期权注销工作，现将相关事项公告如下：',
   '一、股票期权与限制性股票激励计划简述',
   '1、2013年9月2日，公司分别召开了第一届董事会第二十一次会议和第一届监事会第十五次会议，审议通过《三诺生物传感股份有限公司股票期权与限制性股票激励计划（草案）》及其摘要，公司独立董事对此发表了独立意见。',
   '其后公司向中国证监会上报了申请备案材料。',
   '2、根据中国证监会的反馈意见，公司对《三诺生物传感股份有限公司股票期权与限制性股票激励计划（草案）》进行了修订，并于2013年11月18日召开第二届董事会第三次会议和第二届监事会

### 自己测试

In [51]:

#train = read_file("./typed_train.json")
#dev = read_file("./typed_dev.json")
test = read_file("./typed_test.json")
convert_data(test)

ValueError: 

In [47]:
num_sent = []
for i in range(len(test)):
    #for sent in train[i][1]['sentences']:
    num_sent.append(len(test[i][1]['sentences']))
bp.mean(num_sent)    

15.017790262172285